In [1]:
import pandas as pd
import numpy as np
quant_data_path = '/home/sam/data/定增因子.xlsx'
factor_data1 = pd.read_excel(quant_data_path, header=1)

factor_data1.dropna(inplace=True)
tmp = pd.to_datetime(factor_data1['限售股份解禁日']) - pd.to_datetime(factor_data1['发行日期'])
factor_data1['持有天数'] = tmp.apply(lambda x:x.days)
factor_data1['年化收益'] = factor_data1['解禁日绝对收益']*365/factor_data1['持有天数']
factor_data1.drop(columns=['限售股份解禁日','证监会通过公告日', '最新报告期', '调整报告期', '前次报告期', '基金变化', '年度报告期1',\
                       '上市日期', '解禁日收盘价（定点复权）', '沪深300涨跌幅', '解禁日相对沪深300', '中证500涨跌幅',\
                       '解禁日相对中证500', '行业涨跌幅', '解禁日相对行业' ], inplace=True)

factor_data1['发行日期'] = factor_data1['发行日期'].astype(np.string_)
factor_data1['发行月份'] = factor_data1['发行日期'].apply(lambda x:x[:7])

def standardization(quant_col_data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    quant_factor = pd.DataFrame(scaler.fit_transform(quant_col_data),columns=quant_col_data.columns)
    quant_factor.set_index(quant_col_data.index, inplace=True)
    return quant_factor
# 因2020前后因子分布不同，不对折扣率做归一
continu_col = ['发行价格', '预计募资', '总市值', '增发占增发后自由流通市值', '发行前20日股价涨跌幅', \
               '行业20日涨跌幅', '发行前20日相对行业涨跌幅', '股价', '基金占比', '前次占比', '换手率', '闪发', \
               '净利FY0', '净利FY1', '净利FY2', '未来1年净利润增速', '未来2年净利润增速', 'PS百分位',\
               'PE百分位']
factor_data1.replace('-', np.nan, inplace=True)
factor_data1.dropna(inplace=True)
factor_data1[continu_col] = standardization(factor_data1[continu_col])
factor_data1['归一年化收益'] = (factor_data1['年化收益']- factor_data1['年化收益'].mean())/factor_data1['年化收益'].var()
cond = (factor_data1[continu_col] < -3) | (factor_data1[continu_col] > 3)
factor_data1[continu_col].mask(cond, inplace=True)

/tmp/ipykernel_4293/512892657.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factor_data1[continu_col].mask(cond, inplace=True)


In [2]:
factor_data2 = pd.read_excel('/home/sam/data/商业模式涉及财务指标 20221230数据版.xlsx')

In [3]:
factor_data2.drop_duplicates(inplace=True)

In [23]:
factor_data2.isna().sum()

代码                     0
名称                     0
发行日期                   0
最新报告期                  0
年度报告期1                 0
年度报告期2                 0
年度报告期3                 0
总资产增速1                 0
总资产增速2                 0
总资产增速3                 0
货币资金/有息负债1           158
货币资金/有息负债2           244
货币资金/有息负债3           274
固定资产周转率                0
经营性现金流净额/净利润           0
净利润增速                  0
净资产增速                  0
ROE                    0
经营性现金流增速               0
现金收入比                  0
存货周转率                  0
经营杠杆系数                 0
财务杠杆系数                43
经营活动产生的现金流净额/营业收入      0
投入资本回报率1               0
投入资本回报率2               0
投入资本回报率3               0
研发营收比                  0
毛利率1                   0
毛利率2                   0
毛利率3                   0
毛利率波动性                 0
毛利率变异系数               36
成立日期                   0
成立年数                   0
总市值                    0
上市日期                   0
上市年数                   0
经营层持股比例                0
机构持股比例                 0


In [4]:
factor_data2.replace('-', np.nan, inplace=True)
factor_data2['存货周转率'].replace(0, np.nan, inplace=True)
factor_data2.dropna(inplace=True)

In [5]:
factor_data2.drop(columns=['最新报告期', '年度报告期1', '年度报告期2', '年度报告期3', '成立日期','上市日期'], inplace=True)

In [6]:
non_ratio = set(['代码', '名称', '发行日期','固定资产周转率', '现金收入比', '存货周转率', '经营杠杆系数', '财务杠杆系数',\
             '经营活动产生的现金流净额/营业收入', '成立年数', '总市值', '上市年数', '营业收入/长期资本'])
for c in factor_data2.columns:
    if c not in non_ratio:
        factor_data2[c].astype(np.float64)
factor_data2.head(2)

,代码,名称,发行日期,总资产增速1,总资产增速2,总资产增速3,货币资金/有息负债1,货币资金/有息负债2,货币资金/有息负债3,固定资产周转率,...,经营层持股比例,机构持股比例,资产专用度,ROE1,ROE2,ROE3,ROE波动率,营业收入/长期资本,经营资产占比,期间费用率
0,002068.SZ,黑猫股份,2011-01-17,0.292056,0.141513,0.196053,0.248628,0.439818,0.387665,2.4298,...,0.5602,0.779193,0.527701,0.142393,0.005669,0.155255,0.067688,1.250501,0.713355,0.127544
1,600290.SH,ST华仪,2011-01-18,0.573605,0.328553,0.516398,1.148958,0.347032,0.775027,15.3691,...,0.4569,0.039934,0.121788,0.144903,0.200943,0.251324,0.043467,1.121999,0.486928,0.103485


In [7]:
continu_col = factor_data2.columns[3:]
factor_data2[continu_col] = standardization(factor_data2[continu_col])
cond = (factor_data2[continu_col] < -3) | (factor_data2[continu_col] > 3)
factor_data2[continu_col].mask(cond, inplace=True)

/tmp/ipykernel_4293/2598108437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factor_data2[continu_col].mask(cond, inplace=True)


In [8]:
factor_data1.dropna(inplace=True)
factor_data1.drop_duplicates(inplace=True)

In [29]:
factor_data1.head(2)

,代码,名称,发行日期,发行价格,预计募资,增发目的,发行对象,折扣率,总市值,增发占增发后自由流通市值,...,未来1年净利润增速,未来2年净利润增速,PS百分位,PE百分位,大股东是否跟投,解禁日绝对收益,持有天数,年化收益,发行月份,归一年化收益
0,002068.SZ,黑猫股份,2011-01-17,-0.380540,-0.483031,项目融资,机构投资者,-0.182089,-0.530789,0.830084,...,-0.136906,-0.108379,-0.769858,0.469299,否,0.135701,395,0.125394,2011-01,-0.157014
1,600290.SH,ST华仪,2011-01-18,-0.300155,-0.358242,项目融资,"大股东,机构投资者,境内自然人",0.632735,-0.482912,0.370027,...,-0.098737,-0.111949,-1.239744,-1.053054,是,-0.125072,377,-0.121091,2011-01,-0.730488


In [30]:
print(factor_data1.shape)
print(factor_data2.shape)

(1721, 33)
(1750, 42)


In [31]:
for i in range(len(factor_data1)):
    if not factor_data1.iloc[i]['代码'] == factor_data1.iloc[i]['代码']:
        print(factor_data1.iloc[i]['代码'])

In [9]:
factor_data2.drop(columns=['代码', '名称', '发行日期'], inplace=True)

In [10]:
factor_data2.index = range(factor_data2.shape[0])
factor_data1.index = range(factor_data1.shape[0])

In [11]:
factor_data = pd.concat([factor_data1, factor_data2[:1721]], axis=1)

In [12]:
factor_data.to_csv('factor_data.csv', index=False)

In [13]:
from datetime import datetime
start = datetime.strptime('2020-08-10','%Y-%m-%d')
print(factor_data[pd.to_datetime(factor_data['发行日期']) > start].index[0])

1345


In [14]:
train_data = factor_data.loc[:1345]
test_data = factor_data.loc[1345:]
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv',index=False)